## 4. TensorFlow Eager Execution

---
[참고][텐서플로우 블로그]("https://tensorflow.blog/2017/11/01/tensorflow-eager-execution/")

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.enable_eager_execution()

print(tf.__version__)

/Users/Yeoni/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0


### 1) Automatic differentiation and gradient tape

### gradient tapes
---
Trainable variables (created by `tf.Variable` or `tf.get_variable`, where `trainable=True`is default in both cases) are automatically watched.
Tensors can be manually watched by invoking the `watch` method on this context manager.

아래는 미분 예시.

In [2]:
x = tf.constant(1, dtype = tf.float32)

# z = y^2, y = 2x, z = (2x)^2
with tf.GradientTape() as tape:
    tape.watch(x)
    y = tf.add(x, x)
    z = tf.multiply(y, y)
    
# Derivative of z with respect to the original input tensor x
dz_dx = tape.gradient(target = z, sources = x)
print(dz_dx)

tf.Tensor(8.0, shape=(), dtype=float32)


In [3]:
x = tf.constant(1, dtype = tf.float32)

# z = y^2, y = 2x, z = (2x)^2
with tf.GradientTape() as tape:
    tape.watch(x)
    y = tf.add(x, x)
    z = tf.multiply(y, y)
    
# Use the tape to compute the derivative of z with respect to the
# intermediate value y.
dz_dy = tape.gradient(target = z, sources = y)
print(dz_dy)

tf.Tensor(4.0, shape=(), dtype=float32)


In [4]:
x = tf.constant(1, dtype = tf.float32)

# z = y^2, y = 2x, z = (2x)^2
with tf.GradientTape(persistent = True) as tape:
    tape.watch(x)
    y = tf.add(x, x)
    z = tf.multiply(y, y)
    
dz_dy = tape.gradient(target = z, sources = y)
dy_dx = tape.gradient(target = y, sources = x)
dz_dx = tape.gradient(target = z, sources = x)

print(dz_dy, dy_dx, dz_dx)

tf.Tensor(4.0, shape=(), dtype=float32) tf.Tensor(2.0, shape=(), dtype=float32) tf.Tensor(8.0, shape=(), dtype=float32)


바로바로 계산이 가능하지만 다시 만들려면 파이선 세션을 다시 시작해야.. 

In [5]:
def f(x, y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            output = tf.multiply(output, x)
    return output

def grad(x, y):
    with tf.GradientTape() as tape:
        tape.watch(x)
        out = f(x, y)
    return tape.gradient(out, x) 

x = tf.convert_to_tensor(2.0)

print(grad(x, 6)) # out = x^3
print(grad(x, 5)) # out = x^3
print(grad(x, 4)) # out = x^2

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [6]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x * x
    # Compute the gradient inside the 't' context manager
    # which means the gradient computation is differentiable as well.
    dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

print(dy_dx)
print(d2y_dx2)

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


### 2) Variables

In [7]:
# Using python state
x = tf.zeros([10, 10])
x += 2  
        
print(x)

tf.Tensor(
[[2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]], shape=(10, 10), dtype=float32)


In [8]:
v = tf.Variable(1.0)
print(v)

# Re-assign the value
v.assign(3.0)
print(v)

# Use `v` in a TensorFlow operation like tf.square() and reassign
v.assign(tf.square(v))
print(v.numpy)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>
<bound method ResourceVariable.numpy of <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=9.0>>


####  Example: Fitting a linear model
- Define the model
- Define a loss function
- Obtain training data
- Run through the training data and use "optimizer" to adjust the variables to fit the data

In [9]:
#define the model
class Model():
    def __init__(self):
        self.w = tf.Variable(tf.random_normal(shape = []))
        self.b = tf.Variable(0.)
        
    def __call__(self, x):
        return self.w * x + self.b
    
model = Model()

In [10]:
#defien a loss function
def loss_fn(predicted_y, desired_y):
    return tf.reduce_mean(tf.square(predicted_y - desired_y))

In [11]:
# obtain training data

true_w = 3.0
true_b = 2.0
num_examples = 1000

inputs  = tf.random_normal(shape=[num_examples])
noise   = tf.random_normal(shape=[num_examples])
outputs = inputs * true_w + true_b + noise

In [13]:
plt.scatter(inputs, outputs, c='b')
plt.scatter(inputs, model(inputs), c='r')
plt.show()

print('Current loss: '),
print(loss_fn(model(inputs), outputs).numpy())

TypeError: object of type 'tensorflow.python.framework.ops.EagerTensor' has no len()